In [ ]:
import pandas as pd
import numpy as np
from numpy import dstack
import logging
import itertools
import os
import gensim
import nltk
import re

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.utils import to_categorical
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import text, sequence

from nltk.corpus import stopwords



tf.test.gpu_device_name()

In [2]:
fashion = pd.read_csv("Fashion_shuffled.csv")
fashion.tail()


In [3]:
max_words = 1000
batch_size = 3000
epochs = 20


train_size = int(len(fashion) * 0.8) 
train_title = fashion['title'][:train_size] #Takes 70% of the training data to train
train_Category = fashion['Category'][:train_size] #Takes 70% of the training data to train

eval_title = fashion['title'][train_size:] #Takes remaining 30% of the training data to test
eval_Category = fashion['Category'][train_size:] #Takes remaining 30% of the training data to test


tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_title) # only fit on train

x_train = tokenize.texts_to_matrix(train_title)
x_eval = tokenize.texts_to_matrix(eval_title)

encoder = LabelEncoder()
encoder.fit(train_Category)
y_train = train_Category
y_eval = eval_Category
y_train = encoder.transform(train_Category)
y_eval = encoder.transform(eval_Category)

num_classes = np.max(y_train) - np.min(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)   # one hot encode the class values
y_eval = utils.to_categorical(y_eval, num_classes)

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    validation_data=[x_eval, y_eval],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

def display_training_graph(history):
    # summarize history for loss
    plt.plot(history.history['val_loss'])
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['val_loss', 'loss'], loc='upper right')
    plt.draw()


display_training_graph(history)

In [4]:
score = model.evaluate(x_eval, y_eval,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

43941/43941 [==============================] - 1s 18us/step
Test accuracy: 0.6320975812708647


In [5]:
filename = './fashion_models/' + 'model_' + str(1) + '.h5'
model.save(filename)
print('>Saved %s' % filename)

>Saved ./fashion_models/model_1.h5
